In [1]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import os
import pytorch_lightning as pl

from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.loggers import WandbLogger

from ResNet_1 import Net,BasicBlock

In [2]:
import wandb

def Start_wandb(num_epochs=2,architecture="Nice_model"):
    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="FireRed_pixels_classification",

        # track hyperparameters and run metadata
        config={
        #"learning_rate": 0.02,
        "architecture": architecture,
        "dataset": "Pokemon_FireRed_Pixels",
        "epochs": num_epochs,
        }
    )

In [3]:
# Define the data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(16),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.02, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    ]),
    'val': transforms.Compose([
        transforms.Resize(16),
        transforms.CenterCrop(16),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Assuming the images are in a directory named "data", with two subdirectories "train" and "val"
data_dir = 'Data'
image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=50, shuffle=True) for x in ['train', 'val']}




# Init our model
model = Net(BasicBlock, [2,2,2,2])

# Init DataLoader from training set
train_loader = dataloaders['train']


#Using this to store the metrics as a csv for easy to read use
logger_csv = CSVLogger("logs", name=model.name)

In [4]:
#wandb.init(
#        # set the wandb project where this run will be logged
#        project="FireRed_pixels_classification",
#
#        # track hyperparameters and run metadata
#        config={
#        #"learning_rate": 0.02,
#        "architecture": "CNN",
#        "dataset": "Pokemon_FireRed_Pixels",
#        "epochs": 2,
#        }
#    )
Start_wandb(num_epochs=10000,architecture=model.name)
wandb_logger = WandbLogger(save_dir='logs/',name=model.name)
trainer = pl.Trainer(logger=[wandb_logger,logger_csv],max_epochs=10000, devices=1, accelerator="gpu", callbacks=[])
trainer.fit(model,train_loader)
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jaimerbaron. Use `wandb login --relogin` to force relogin


c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.h

Training: 0it [00:00, ?it/s]

c:\Users\Rafa\miniconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
c:\Users\Rafa\miniconda3\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in macro-average
  warnings.warn(*args, **kwargs)
c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:212: UserWarning: You called `self.log('train_mcc', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\call.py:44 in               │
│ _call_and_handle_interrupt                                                                       │
│                                                                                                  │
│    41 │   │   if trainer.strategy.launcher is not None:                                          │
│    42 │   │   │   return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer,    │
│    43 │   │   else:                                                                              │
│ ❱  44 │   │   │   return trainer_fn(*args, **kwargs)                                             │
│    45 │                                                                                          │
│    46 │   except _TunerExitException:                                                            │
│    47 │   │   _call_teardown_hook(trainer)                                                       │
│                                                                                                  │
│ c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:559 in _fit_impl │
│                                                                                                  │
│    556 │   │   │   model_provided=True,                                                          │
│    557 │   │   │   model_connected=self.lightning_module is not None,                            │
│    558 │   │   )                                                                                 │
│ ❱  559 │   │   self._run(model, ckpt_path=ckpt_path)                                             │
│    560 │   │                                                                                     │
│    561 │   │   assert self.state.stopped                                                         │
│    562 │   │   self.training = False                                                             │
│                                                                                                  │
│ c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:935 in _run      │
│                                                                                                  │
│    932 │   │   # ----------------------------                                                    │
│    933 │   │   # RUN THE TRAINER                                                                 │
│    934 │   │   # ----------------------------                                                    │
│ ❱  935 │   │   results = self._run_stage()                                                       │
│    936 │   │                                                                                     │
│    937 │   │   # ----------------------------                                                    │
│    938 │   │   # POST-Training CLEAN UP                                                          │
│                                                                                                  │
│ c:\Users\Rafa\miniconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:978 in           │
│ _run_stage                                                                                       │
│                                                                                                  │
│    975 │   │   │   with isolate_rng():                                                           │
│    976 │   │   │   │   self._run_sanity_check()                                                  │
│    977 │   │   │   with torch.autograd.set_detect_anomaly(self._detect_anomaly):                 │
│ ❱  978 │   │   │   │   self.fit_loop.run()                                                       │
│    979 │   │   │   return None                             